In [10]:
%run ./cs_go_import_and_process_data.ipynb

In [24]:
# sniper analysis

In [26]:
(df_teams
  .groupBy('snipers_on_team')
  .agg(F.count('snipers_on_team'))
).show()

+------------------+-------------------------+
|t1_snipers_on_team|count(t1_snipers_on_team)|
+------------------+-------------------------+
|                 1|                     5516|
|                 3|                       61|
|                 2|                     1825|
|                 0|                      172|
+------------------+-------------------------+



In [142]:
df_matches_by_number_of_snipers = (df_matches_exploded
    .groupBy('team_name')
    .pivot('t1_snipers_on_team')
    .agg(F.count('team_name'))
    .na.fill(0)
    .withColumn('m_ttl',
        F.col('0') + F.col('1') + F.col('2') + F.col('3') 
    )
    .select(
        'team_name',
        F.col('0').alias('m_0'),
        F.col('1').alias('m_1'),
        F.col('2').alias('m_2'),
        F.col('3').alias('m_3'),
        'm_ttl',
    )
    .orderBy(F.col('m_ttl').desc())
)

df_matches_by_number_of_snipers.show(50)

+--------------+---+---+---+---+-----+
|     team_name|m_0|m_1|m_2|m_3|m_ttl|
+--------------+---+---+---+---+-----+
|      ASTRALIS| 13|440| 73|  0|  526|
|          FAZE|  0|439| 66|  0|  505|
|   MOUSESPORTS|  0|196|264| 12|  472|
|           NIP| 71|218|174|  0|  463|
|            G2|  0|276|183|  0|  459|
|        FNATIC|  0|308| 97| 12|  417|
| NATUS VINCERE|  0|271|121|  4|  396|
|        LIQUID|  6|258|108|  6|  378|
|    VIRTUS.PRO|  0| 69|188| 23|  280|
|        CLOUD9|  0|211| 55|  0|  266|
|        HEROIC| 18|207| 31|  0|  256|
|         NORTH|  8|230| 10|  0|  248|
|            SK|  0|206| 34|  0|  240|
|          ENVY|  0|198| 12|  0|  210|
|   HELLRAISERS|  0|160| 17|  0|  177|
|         OPTIC|  0| 91| 69|  0|  160|
|        GAMBIT|  0|156|  1|  0|  157|
|           BIG|  0|102| 42|  0|  144|
|      DIGNITAS| 22|114|  1|  0|  137|
|     RENEGADES|  0|131|  1|  0|  132|
|      VITALITY|  0|117| 13|  0|  130|
|          MIBR|  0| 70| 57|  0|  127|
|     IMMORTALS|  0| 73| 

In [146]:
df_wins_by_number_of_snipers = (df_matches_exploded
    .groupBy('team_name')
    .pivot('t1_snipers_on_team')
    .agg(F.sum(F.when(F.col('won') == True, F.lit(1)).otherwise(F.lit(0))).alias('wins'))
    .na.fill(0)
    .withColumn('w_ttl',
        F.col('0') + F.col('1') + F.col('2') + F.col('3') 
    )
    .select(
        'team_name',
        F.col('0').alias('w_0'),
        F.col('1').alias('w_1'),
        F.col('2').alias('w_2'),
        F.col('3').alias('w_3'),
        'w_ttl',
    )
    .orderBy(F.col('w_ttl').desc())
)

# df_wins_by_number_of_snipers.show(50)

+--------------+---+---+---+---+-----+
|     team_name|w_0|w_1|w_2|w_3|w_ttl|
+--------------+---+---+---+---+-----+
|      ASTRALIS|  6|307| 39|  0|  352|
|          FAZE|  0|237| 29|  0|  266|
|   MOUSESPORTS|  0|102|138|  9|  249|
|           NIP| 30|118| 85|  0|  233|
|        FNATIC|  0|173| 47|  4|  224|
| NATUS VINCERE|  0|144| 75|  4|  223|
|            G2|  0|129| 91|  0|  220|
|        LIQUID|  4|137| 54|  6|  201|
|        CLOUD9|  0|111| 29|  0|  140|
|            SK|  0|120| 20|  0|  140|
|         NORTH|  7|120|  5|  0|  132|
|    VIRTUS.PRO|  0| 26| 76| 11|  113|
|        HEROIC|  6| 81| 18|  0|  105|
|          ENVY|  0| 81|  8|  0|   89|
|      VITALITY|  0| 68| 11|  0|   79|
|   HELLRAISERS|  0| 68|  6|  0|   74|
|        GAMBIT|  0| 69|  0|  0|   69|
|           BIG|  0| 44| 24|  0|   68|
|      DIGNITAS|  6| 57|  0|  0|   63|
|         OPTIC|  0| 31| 31|  0|   62|
|     IMMORTALS|  0| 36| 20|  0|   56|
|          MIBR|  0| 31| 22|  0|   53|
|         FURIA|  0| 17| 

In [159]:
df_matches_and_wins_by_number_of_snipers = (df_matches_by_number_of_snipers
    .join(df_wins_by_number_of_snipers, 'team_name', 'left')
    .select(
        '*',
        F.lit(F.round(F.col('w_0') / F.col('m_0'), 2)).alias('p_0'),
        F.lit(F.round(F.col('w_1') / F.col('m_1'), 2)).alias('p_1'),
        F.lit(F.round(F.col('w_2') / F.col('m_2'), 2)).alias('p_2'),
        F.lit(F.round(F.col('w_3') / F.col('m_3'), 2)).alias('p_3'),
    )
    .na.fill(0)
)

df_matches_and_wins_by_number_of_snipers.show(80)

+--------------------+---+---+---+---+-----+---+---+---+---+-----+----+----+----+----+
|           team_name|m_0|m_1|m_2|m_3|m_ttl|w_0|w_1|w_2|w_3|w_ttl| p_0| p_1| p_2| p_3|
+--------------------+---+---+---+---+-----+---+---+---+---+-----+----+----+----+----+
|            DIGNITAS| 22|114|  1|  0|  137|  6| 57|  0|  0|   63|0.27| 0.5| 0.0| 0.0|
|               CR4ZY|  0| 23|  3|  0|   26|  0| 11|  1|  0|   12| 0.0|0.48|0.33| 0.0|
|                 CLG|  0| 28| 44|  0|   72|  0| 10| 10|  0|   20| 0.0|0.36|0.23| 0.0|
|             C0NTACT|  0|  1|  0|  0|    1|  0|  0|  0|  0|    0| 0.0| 0.0| 0.0| 0.0|
|                 TSM|  0|  1|  0|  0|    1|  0|  0|  0|  0|    0| 0.0| 0.0| 0.0| 0.0|
|              SPIRIT|  0| 26|  0|  0|   26|  0| 10|  0|  0|   10| 0.0|0.38| 0.0| 0.0|
|       EVIL GENIUSES|  0| 71|  0|  0|   71|  0| 39|  0|  0|   39| 0.0|0.55| 0.0| 0.0|
|       NATUS VINCERE|  0|271|121|  4|  396|  0|144| 75|  4|  223| 0.0|0.53|0.62| 1.0|
|              FNATIC|  0|308| 97| 12|  417

In [27]:
sniper_counts_by_match_winning_teams = (df_player_stats_exploded
    .filter(F.col('won') == True)
    .groupBy('match_id')
    .agg(F.sum(F.when(F.col('t1_player1_is_sniper') == True, 1)).alias('number_of_snipers'))
)

snipers_per_winning_team = (sniper_counts_by_match_winning_teams
    .groupBy('number_of_snipers')
    .agg(F.count('match_id'))
    .orderBy('number_of_snipers')
)

snipers_per_winning_team.show()

+-----------------+---------------+
|number_of_snipers|count(match_id)|
+-----------------+---------------+
|             null|              5|
|                1|            122|
|                2|           1028|
|                3|            591|
|                4|            162|
|                5|             12|
|                6|              1|
+-----------------+---------------+



In [68]:
sniper_counts_by_match_losing_teams = (df_player_stats_exploded
    .filter(F.col('won') == False)
    .groupBy('match_id')
    .agg(F.sum(F.when(F.col('t1_player1_is_sniper') == True, 1)).alias('number_of_snipers'))
)

snipers_per_winning_team = (sniper_counts_by_match_losing_teams
    .groupBy('number_of_snipers')
    .agg(F.count('match_id'))
    .orderBy('number_of_snipers')
)

snipers_per_winning_team.show()

+-----------------+---------------+
|number_of_snipers|count(match_id)|
+-----------------+---------------+
|             null|              3|
|                1|            133|
|                2|           1026|
|                3|            638|
|                4|            141|
|                5|              3|
|                6|              1|
+-----------------+---------------+



In [ ]:
# Underdog analysis

In [104]:
df_csgo_with_underdog_check = (df_csgo
    # team ranking
    .withColumn('favourite_by_ranking',
        F.when(F.col('t1_world_rank') > F.col('t2_world_rank'), F.lit('t1'))
         .when(F.col('t2_world_rank') > F.col('t1_world_rank'), F.lit('t2'))
         .otherwise(F.lit('even'))      
    )
    .withColumn('ranking_difference',
        F.abs(F.col('t1_world_rank') - F.col('t2_world_rank'))
    )
    .withColumn('winner_by_comparitive_ranking',
        F.when(F.col('favourite_by_ranking') == F.col('winner'), F.lit('favourite'))
         .otherwise(F.lit('underdog'))
    )
    
    # h2h
    .withColumn('favourite_by_h2h',
        F.when(F.col('t1_h2h_win_perc') > F.col('t2_h2h_win_perc'), F.lit('t1'))
         .when(F.col('t2_h2h_win_perc') > F.col('t1_h2h_win_perc'), F.lit('t2'))
         .otherwise(F.lit('even'))      
    )
    .withColumn('h2h_difference',
        F.abs(F.col('t1_h2h_win_perc') - F.col('t2_h2h_win_perc'))
    )
    .withColumn('winner_by_h2h',
        F.when(F.col('favourite_by_h2h') == F.col('winner'), F.lit('favourite'))
         .otherwise(F.lit('underdog'))
    )
     
    # avg player rating
    .withColumn('favourite_by_avg_player_rating',
        F.when(F.col('t1_avg_player_rating') > F.col('t2_avg_player_rating'), F.lit('t1'))
         .when(F.col('t2_avg_player_rating') > F.col('t1_avg_player_rating'), F.lit('t2'))
         .otherwise(F.lit('even'))      
    )
    .withColumn('avg_player_rating_difference',
        F.abs(F.col('t1_h2h_win_perc') - F.col('t2_h2h_win_perc'))
    )
    .withColumn('winner_by_avg_player_rating',
        F.when(F.col('favourite_by_avg_player_rating') == F.col('winner'), F.lit('favourite'))
         .otherwise(F.lit('underdog'))
    )
)

In [81]:
(df_csgo_with_underdog_check
    .groupBy('winner_by_comparitive_ranking')
    .agg(F.count('winner_by_comparitive_ranking'))
    .show()
)

+-----------------------------+------------------------------------+
|winner_by_comparitive_ranking|count(winner_by_comparitive_ranking)|
+-----------------------------+------------------------------------+
|                    favourite|                                1542|
|                     underdog|                                2245|
+-----------------------------+------------------------------------+



In [76]:
(df_csgo_with_underdog_check
    .groupBy('ranking_difference')
    .agg(F.count('ranking_difference'))
    .orderBy('ranking_difference')
    .show(40)
)

+------------------+-------------------------+
|ranking_difference|count(ranking_difference)|
+------------------+-------------------------+
|                 1|                      448|
|                 2|                      418|
|                 3|                      390|
|                 4|                      330|
|                 5|                      315|
|                 6|                      301|
|                 7|                      239|
|                 8|                      251|
|                 9|                      232|
|                10|                      178|
|                11|                      147|
|                12|                      143|
|                13|                      123|
|                14|                       93|
|                15|                       48|
|                16|                       49|
|                17|                       42|
|                18|                       21|
|            

In [79]:
(df_csgo_with_underdog_check
    .groupBy('ranking_difference')
    .pivot('winner_by_comparitive_ranking')
    .agg(F.count('winner_by_comparitive_ranking'))
    .orderBy('ranking_difference')
    .show(40)
)

+------------------+---------+--------+
|ranking_difference|favourite|underdog|
+------------------+---------+--------+
|                 1|      194|     254|
|                 2|      205|     213|
|                 3|      161|     229|
|                 4|      147|     183|
|                 5|      138|     177|
|                 6|      111|     190|
|                 7|       87|     152|
|                 8|       99|     152|
|                 9|       94|     138|
|                10|       63|     115|
|                11|       47|     100|
|                12|       51|      92|
|                13|       55|      68|
|                14|       37|      56|
|                15|       17|      31|
|                16|       13|      36|
|                17|       12|      30|
|                18|        6|      15|
|                19|        5|      13|
|                20|     null|       1|
+------------------+---------+--------+



In [82]:
(df_csgo_with_underdog_check
    .groupBy('winner_by_h2h')
    .agg(F.count('winner_by_h2h'))
    .show()
)

+-------------+--------------------+
|winner_by_h2h|count(winner_by_h2h)|
+-------------+--------------------+
|    favourite|                1807|
|     underdog|                1980|
+-------------+--------------------+



In [105]:
(df_csgo_with_underdog_check
    .groupBy('winner_by_avg_player_rating')
    .agg(F.count('winner_by_avg_player_rating'))
    .show()
)

+---------------------------+----------------------------------+
|winner_by_avg_player_rating|count(winner_by_avg_player_rating)|
+---------------------------+----------------------------------+
|                  favourite|                              2163|
|                   underdog|                              1624|
+---------------------------+----------------------------------+

